In [1]:
%matplotlib inline
from os.path import join as opj
import os
import json
import time
import numpy as np
import nibabel as nib

from nipype.interfaces.fsl import BET
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.pipeline.engine import Workflow, Node
from nipype import SelectFiles

from nipype.interfaces.ants import Registration
from nipype.interfaces.ants.segmentation import BrainExtraction
from nipype.testing import example_data

import pandas as pd

from utils import load_nifti

import matplotlib.pyplot as plt

In [12]:
#root_dir = '/analysis/fabiane/data/ADNI_subset/'
root_dir = '/analysis/share/ADNI'
data_dir = os.path.join(root_dir, 'ADNI_2Yr_15')

#subjects = ['ADNI_023_S_0058', 'ADNI_023_S_0061', 'ADNI_136_S_0300', 'ADNI_136_S_0426']
#subjects = ['051_S_1331', '027_S_1082']#, '023_S_0030']

template_dir = '/analysis/share/Ritter/Templates/'

ants_settings = '/home/fabiane/anaconda2/pkgs/nipype-0.14.0-py36_1/lib/python3.6/site-packages/nipype/testing/data/smri_ants_registration_settings.json'

### Prepare Data

In [13]:
# Read patient list
df = pd.read_csv(os.path.join(root_dir,
                              'ADNI_tables',
                              'customized',
                              'ClassDxByImg_CompleteAnnual2YearVisitList_1_5T.csv'
                              ))


In [14]:
# Filter unique patient IDs
subjects = list(df['PTID'].unique())

### Prepare Nodes

In [5]:
# Define number of threads per scan
thread_count = 6

In [6]:
# BET - Skullstripping (FSL)
# Node not currently in use!
bet = Node(BET(frac=0.3,
               #robust=True,
               reduce_bias=True,
               mask=True,
               output_type='NIFTI_GZ'),
           name="Skullstripping")

In [7]:
# ANTS - Brain Extraction
# Node not currently in use!
bet = Node(BrainExtraction(dimension=3,
                           brain_template=os.path.join(template_dir,
                                                       'mni_icbm152_nlin_asym_09c',
                                                       'mni_icbm152_t1_tal_nlin_asym_09c.nii'
                                                       ),
                           brain_probability_mask=os.path.join(
                                                       template_dir,
                                                       'mni_icbm152_nlin_asym_09c',
                                                       'mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'),
                          num_threads=thread_count),
           name="Skullstripping"
           )

In [8]:
# ANTS - Registration 
reg = Node(Registration(from_file=example_data('smri_ants_registration_settings.json'),
                        num_threads=thread_count),
           name="Registration")

#reg.inputs.fixed_image = os.path.join(template_dir, 'T1.nii')
reg.inputs.fixed_image = os.path.join(template_dir,
                                   'mni_icbm152_nlin_asym_09c',
                                   'mni_icbm152_t1_tal_nlin_asym_09c.nii')

In [9]:
print(reg.inputs)


args = <undefined>
collapse_output_transforms = False
convergence_threshold = [1e-08, 1e-08, 1e-08, -0.01]
convergence_window_size = [20, 20, 20, 5]
dimension = 3
environ = {'NSLOTS': '6'}
fixed_image = ['/analysis/share/Ritter/Templates/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii']
fixed_image_mask = <undefined>
fixed_image_masks = <undefined>
float = <undefined>
ignore_exception = False
initial_moving_transform = <undefined>
initial_moving_transform_com = True
initialize_transforms_per_stage = False
interpolation = Linear
interpolation_parameters = <undefined>
invert_initial_moving_transform = <undefined>
metric = ['Mattes', 'Mattes', 'Mattes', ['Mattes', 'CC']]
metric_item_trait = <undefined>
metric_stage_trait = <undefined>
metric_weight = [1.0, 1.0, 1.0, [0.5, 0.5]]
metric_weight_item_trait = <undefined>
metric_weight_stage_trait = <undefined>
moving_image = <undefined>
moving_image_mask = <undefined>
moving_image_masks = <undefined>
num_threads = 6
number_of_i

In [15]:
# Infosource for parallel processing
infosource = Node(interface=IdentityInterface(
                     fields=['subject_scan', 'subject_id']),
                     name="infosource"
                 )
infosource.iterables = [('subject_scan', ['Screening', 'Month12', 'Month24']),
                        ('subject_id', subjects)
                        ]


In [16]:
# Datasource which selects the files
templates = dict(struct='{subject_id}/{subject_scan}/Structural/{filename}*.nii')
datasource = Node(SelectFiles(templates), name='SelectFiles')
datasource.inputs.base_directory = os.path.abspath(data_dir)

In [17]:
# Datasink
datasink = Node(interface=DataSink(), name="DataSink")
datasink.inputs.substitutions = [('_subject_id_', ''),
                                 ('_subject_scan_', '/')
                                 #('/INTERNAL_WARPED', '')
                                ]
datasink.inputs.base_directory = root_dir

### Connect Workflow

In [18]:
#preprocessing = Workflow(base_dir=root_dir, name="preprocessing")

#preprocessing.connect(infosource, "subject_id", datasource, "subject_id")
#preprocessing.connect(infosource, "subject_scan", datasource, "subject_scan")
#preprocessing.connect(infosource, "subject_id", datasource, "filename")
#preprocessing.connect(datasource, "struct", reg, "moving_image")
#preprocessing.connect(reg, "warped_image", datasink, "working_dir")

In [19]:
# Connect the different workflow parts
preprocessing = Workflow(base_dir=root_dir, name="preprocessing")
# Pass subject information for the datasource template
preprocessing.connect(infosource, "subject_id", datasource, "subject_id")
preprocessing.connect(infosource, "subject_scan", datasource, "subject_scan")
preprocessing.connect(infosource, "subject_id", datasource, "filename")
preprocessing.connect(datasource, "struct", reg, "moving_image")
# final datasink parameter is the output folder
preprocessing.connect(reg, "warped_image", datasink, "preprocessed_1_5T") 

In [20]:
# DEBUG VERSION
#preprocessing = Workflow(base_dir=root_dir, name="preprocessing")

#preprocessing.connect(infosource, "subject_id", datasource, "subject_id")
#preprocessing.connect(infosource, "subject_scan", datasource, "subject_scan")
#preprocessing.connect(infosource, "subject_id", datasource, "filename")
#preprocessing.connect(datasource, "struct", bet, "anatomical_image")
#preprocessing.connect(bet, "BrainExtractionBrain", reg, "moving_image")
#preprocessing.connect(datasource, "struct", reg, "moving_image")
#preprocessing.connect(reg, "warped_image", datasink, "working_dir.with_skull")

In [21]:
# Start the preprocessing using the IpyParallel profile grid
start_time = time.time()
preprocessing.run(plugin='IPython', plugin_args={'profile' :'grid'})
print('Finished preprocessing after {} minutes'.format((time.time() - start_time)/60))

180201-16:27:11,295 workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
180201-16:27:27,752 workflow INFO:
	 Running in parallel.
180201-16:27:28,20 workflow INFO:
	 Pending[0] Submitting[1512] jobs Slots[99999]
180201-16:27:28,21 workflow INFO:
	 Submitting: SelectFiles.a1511 ID: 0
180201-16:27:28,26 workflow INFO:
	 Finished submitting: SelectFiles.a1511 ID: 0
180201-16:27:28,29 workflow INFO:
	 Submitting: SelectFiles.a1510 ID: 3
180201-16:27:28,36 workflow INFO:
	 Finished submitting: SelectFiles.a1510 ID: 3
180201-16:27:28,40 workflow INFO:
	 Submitting: SelectFiles.a1509 ID: 6
180201-16:27:28,48 workflow INFO:
	 Finished submitting: SelectFiles.a1509 ID: 6
180201-16:27:28,50 workflow INFO:
	 Submitting: SelectFiles.a1508 ID: 9
180201-16:27:28,56 workflow INFO:
	 Finished submitting: SelectFiles.a1508 ID: 9
180201-16:27:28,60 workflow INFO:
	 Submitting: SelectFiles.a1507 ID: 12
180201-16:27:28,66 workflow INFO:
	 Finished submitting: 

180201-16:27:28,480 workflow INFO:
	 Submitting: SelectFiles.a1460 ID: 153
180201-16:27:28,485 workflow INFO:
	 Finished submitting: SelectFiles.a1460 ID: 153
180201-16:27:28,487 workflow INFO:
	 Submitting: SelectFiles.a1459 ID: 156
180201-16:27:28,492 workflow INFO:
	 Finished submitting: SelectFiles.a1459 ID: 156
180201-16:27:28,494 workflow INFO:
	 Submitting: SelectFiles.a1458 ID: 159
180201-16:27:28,498 workflow INFO:
	 Finished submitting: SelectFiles.a1458 ID: 159
180201-16:27:28,499 workflow INFO:
	 Submitting: SelectFiles.a1457 ID: 162
180201-16:27:28,505 workflow INFO:
	 Finished submitting: SelectFiles.a1457 ID: 162
180201-16:27:28,506 workflow INFO:
	 Submitting: SelectFiles.a1456 ID: 165
180201-16:27:28,510 workflow INFO:
	 Finished submitting: SelectFiles.a1456 ID: 165
180201-16:27:28,512 workflow INFO:
	 Submitting: SelectFiles.a1455 ID: 168
180201-16:27:28,526 workflow INFO:
	 Finished submitting: SelectFiles.a1455 ID: 168
180201-16:27:28,528 workflow INFO:
	 Submittin

180201-16:27:28,805 workflow INFO:
	 Submitting: SelectFiles.a1408 ID: 309
180201-16:27:28,808 workflow INFO:
	 Finished submitting: SelectFiles.a1408 ID: 309
180201-16:27:28,809 workflow INFO:
	 Submitting: SelectFiles.a1407 ID: 312
180201-16:27:28,812 workflow INFO:
	 Finished submitting: SelectFiles.a1407 ID: 312
180201-16:27:28,814 workflow INFO:
	 Submitting: SelectFiles.a1406 ID: 315
180201-16:27:28,819 workflow INFO:
	 Finished submitting: SelectFiles.a1406 ID: 315
180201-16:27:28,820 workflow INFO:
	 Submitting: SelectFiles.a1405 ID: 318
180201-16:27:28,823 workflow INFO:
	 Finished submitting: SelectFiles.a1405 ID: 318
180201-16:27:28,825 workflow INFO:
	 Submitting: SelectFiles.a1404 ID: 321
180201-16:27:28,829 workflow INFO:
	 Finished submitting: SelectFiles.a1404 ID: 321
180201-16:27:28,830 workflow INFO:
	 Submitting: SelectFiles.a1403 ID: 324
180201-16:27:28,834 workflow INFO:
	 Finished submitting: SelectFiles.a1403 ID: 324
180201-16:27:28,835 workflow INFO:
	 Submittin

180201-16:27:29,293 workflow INFO:
	 Submitting: SelectFiles.a1356 ID: 465
180201-16:27:29,311 workflow INFO:
	 Finished submitting: SelectFiles.a1356 ID: 465
180201-16:27:29,314 workflow INFO:
	 Submitting: SelectFiles.a1355 ID: 468
180201-16:27:29,344 workflow INFO:
	 Finished submitting: SelectFiles.a1355 ID: 468
180201-16:27:29,348 workflow INFO:
	 Submitting: SelectFiles.a1354 ID: 471
180201-16:27:29,363 workflow INFO:
	 Finished submitting: SelectFiles.a1354 ID: 471
180201-16:27:29,369 workflow INFO:
	 Submitting: SelectFiles.a1353 ID: 474
180201-16:27:29,386 workflow INFO:
	 Finished submitting: SelectFiles.a1353 ID: 474
180201-16:27:29,390 workflow INFO:
	 Submitting: SelectFiles.a1352 ID: 477
180201-16:27:29,402 workflow INFO:
	 Finished submitting: SelectFiles.a1352 ID: 477
180201-16:27:29,405 workflow INFO:
	 Submitting: SelectFiles.a1351 ID: 480
180201-16:27:29,418 workflow INFO:
	 Finished submitting: SelectFiles.a1351 ID: 480
180201-16:27:29,422 workflow INFO:
	 Submittin

180201-16:27:29,886 workflow INFO:
	 Submitting: SelectFiles.a1304 ID: 621
180201-16:27:29,902 workflow INFO:
	 Finished submitting: SelectFiles.a1304 ID: 621
180201-16:27:29,905 workflow INFO:
	 Submitting: SelectFiles.a1303 ID: 624
180201-16:27:29,913 workflow INFO:
	 Finished submitting: SelectFiles.a1303 ID: 624
180201-16:27:29,914 workflow INFO:
	 Submitting: SelectFiles.a1302 ID: 627
180201-16:27:29,922 workflow INFO:
	 Finished submitting: SelectFiles.a1302 ID: 627
180201-16:27:29,924 workflow INFO:
	 Submitting: SelectFiles.a1301 ID: 630
180201-16:27:29,928 workflow INFO:
	 Finished submitting: SelectFiles.a1301 ID: 630
180201-16:27:29,930 workflow INFO:
	 Submitting: SelectFiles.a1300 ID: 633
180201-16:27:29,935 workflow INFO:
	 Finished submitting: SelectFiles.a1300 ID: 633
180201-16:27:29,936 workflow INFO:
	 Submitting: SelectFiles.a1299 ID: 636
180201-16:27:29,944 workflow INFO:
	 Finished submitting: SelectFiles.a1299 ID: 636
180201-16:27:29,945 workflow INFO:
	 Submittin

...

180201-16:28:26,93 workflow INFO:
	 Finished submitting: Registration.a0856 ID: 1966
180201-16:28:26,95 workflow INFO:
	 Submitting: Registration.a0855 ID: 1969
180201-16:28:26,106 workflow INFO:
	 Finished submitting: Registration.a0855 ID: 1969
180201-16:28:26,107 workflow INFO:
	 Submitting: Registration.a0854 ID: 1972
180201-16:28:26,119 workflow INFO:
	 Finished submitting: Registration.a0854 ID: 1972
180201-16:28:26,121 workflow INFO:
	 Submitting: Registration.a0853 ID: 1975
180201-16:28:26,131 workflow INFO:
	 Finished submitting: Registration.a0853 ID: 1975
180201-16:28:26,133 workflow INFO:
	 Submitting: Registration.a0852 ID: 1978
180201-16:28:26,143 workflow INFO:
	 Finished submitting: Registration.a0852 ID: 1978
180201-16:28:26,147 workflow INFO:
	 Submitting: Registration.a0850 ID: 1984
180201-16:28:26,156 workflow INFO:
	 Finished submitting: Registration.a0850 ID: 1984
180201-16:28:26,157 workflow INFO:
	 Submitting: Registration.a0849 ID: 1987
180201-16:28:26,168 work

180201-16:28:26,700 workflow INFO:
	 Submitting: Registration.a0800 ID: 2134
180201-16:28:26,708 workflow INFO:
	 Finished submitting: Registration.a0800 ID: 2134
180201-16:28:26,710 workflow INFO:
	 Submitting: Registration.a0798 ID: 2140
180201-16:28:26,721 workflow INFO:
	 Finished submitting: Registration.a0798 ID: 2140
180201-16:28:26,722 workflow INFO:
	 Submitting: Registration.a0797 ID: 2143
180201-16:28:26,735 workflow INFO:
	 Finished submitting: Registration.a0797 ID: 2143
180201-16:28:26,738 workflow INFO:
	 Submitting: Registration.a0796 ID: 2146
180201-16:28:26,749 workflow INFO:
	 Finished submitting: Registration.a0796 ID: 2146
180201-16:28:26,750 workflow INFO:
	 Submitting: Registration.a0795 ID: 2149
180201-16:28:26,759 workflow INFO:
	 Finished submitting: Registration.a0795 ID: 2149
180201-16:28:26,761 workflow INFO:
	 Submitting: Registration.a0794 ID: 2152
180201-16:28:26,770 workflow INFO:
	 Finished submitting: Registration.a0794 ID: 2152
180201-16:28:26,771 wo

180201-16:28:27,306 workflow INFO:
	 Finished submitting: Registration.a0745 ID: 2299
180201-16:28:27,307 workflow INFO:
	 Submitting: Registration.a0744 ID: 2302
180201-16:28:27,317 workflow INFO:
	 Finished submitting: Registration.a0744 ID: 2302
180201-16:28:27,319 workflow INFO:
	 Submitting: Registration.a0743 ID: 2305
180201-16:28:27,329 workflow INFO:
	 Finished submitting: Registration.a0743 ID: 2305
180201-16:28:27,331 workflow INFO:
	 Submitting: Registration.a0742 ID: 2308
180201-16:28:27,340 workflow INFO:
	 Finished submitting: Registration.a0742 ID: 2308
180201-16:28:27,341 workflow INFO:
	 Submitting: Registration.a0741 ID: 2311
180201-16:28:27,351 workflow INFO:
	 Finished submitting: Registration.a0741 ID: 2311
180201-16:28:27,352 workflow INFO:
	 Submitting: Registration.a0740 ID: 2314
180201-16:28:27,364 workflow INFO:
	 Finished submitting: Registration.a0740 ID: 2314
180201-16:28:27,366 workflow INFO:
	 Submitting: Registration.a0739 ID: 2317
180201-16:28:27,375 wo

180201-16:28:27,886 workflow INFO:
	 Submitting: Registration.a0692 ID: 2458
180201-16:28:27,899 workflow INFO:
	 Finished submitting: Registration.a0692 ID: 2458
180201-16:28:27,900 workflow INFO:
	 Submitting: Registration.a0691 ID: 2461
180201-16:28:27,911 workflow INFO:
	 Finished submitting: Registration.a0691 ID: 2461
180201-16:28:27,912 workflow INFO:
	 Submitting: Registration.a0690 ID: 2464
180201-16:28:27,922 workflow INFO:
	 Finished submitting: Registration.a0690 ID: 2464
180201-16:28:27,923 workflow INFO:
	 Submitting: Registration.a0689 ID: 2467
180201-16:28:27,932 workflow INFO:
	 Finished submitting: Registration.a0689 ID: 2467
180201-16:28:27,933 workflow INFO:
	 Submitting: Registration.a0688 ID: 2470
180201-16:28:27,942 workflow INFO:
	 Finished submitting: Registration.a0688 ID: 2470
180201-16:28:27,944 workflow INFO:
	 Submitting: Registration.a0687 ID: 2473
180201-16:28:27,953 workflow INFO:
	 Finished submitting: Registration.a0687 ID: 2473
180201-16:28:27,955 wo

TraitError: The trait 'moving_image' of a RegistrationInputSpec instance is an existing file name, but the path  '/analysis/share/ADNI/ADNI_2Yr_15T/023_S_0376/Month12/Structural/023_S_0376_3-20-07_Month12_64791_MCI.nii' does not exist.
Error setting node input:
Node: Registration
input: moving_image
results_file: /analysis/share/ADNI/preprocessing/_subject_id_023_S_0376_subject_scan_Month12/SelectFiles/result_SelectFiles.pklz
value: ['/analysis/share/ADNI/ADNI_2Yr_15T/023_S_0376/Month12/Structural/023_S_0376_3-20-07_Month12_64791_MCI.nii', '/analysis/share/ADNI/ADNI_2Yr_15T/023_S_0376/Month12/Structural/023_S_0376_3-20-07_Month12_118844_MCI.nii']